# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-09-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-09-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-09-10 04:29:01,33.93911,67.709953,38544,1420,31048,6076.0,Afghanistan,99.012696,3.684101
1,NaN,NaN,NaN,Albania,2020-09-10 04:29:01,41.15330,20.168300,10704,322,6284,4098.0,Albania,371.950796,3.008221
2,NaN,NaN,NaN,Algeria,2020-09-10 04:29:01,28.03390,1.659600,47216,1581,33379,12256.0,Algeria,107.673608,3.348441
3,NaN,NaN,NaN,Andorra,2020-09-10 04:29:01,42.50630,1.521800,1301,53,938,310.0,Andorra,1683.815440,4.073789
4,NaN,NaN,NaN,Angola,2020-09-10 04:29:01,-11.20270,17.873900,3092,126,1245,1721.0,Angola,9.407822,4.075032


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38165,38196,38243,38288,38304,38324,38398,38494,38520,38544
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9513,9606,9728,9844,9967,10102,10255,10406,10553,10704
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,44494,44833,45158,45469,45773,46071,46364,46653,46938,47216


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1402,1406,1409,1409,1409,1409,1412,1415,1418,1420
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,284,290,296,301,306,312,316,319,321,322
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1510,1518,1523,1529,1539,1549,1556,1562,1571,1581


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,29089,29231,29315,29390,29713,30082,30537,30557,30715,31048
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,5214,5441,5582,5732,5882,5976,6106,6186,6239,6284
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,31244,31493,31746,32006,32259,32481,32745,32985,33183,33379


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1348,1354,1345,1349,1355,1371,1377,1383,1385,1398
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4422,4445,4458,4495,4513,4542,4569,4586,4609,4639
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,628,629,617,614,616,616,616,617,617,618


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,22,23,23,23,23,23,23,23,23,23
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,38,38,38,40,42,42,42,42,42,42
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-09-10 04:29:01,32.539527,-86.644082,1398,23,0,1375.0,"Autauga, Alabama, US",2502.282124,1.645207
668,1075.0,Lamar,Alabama,US,2020-09-10 04:29:01,33.779950,-88.096680,282,2,0,280.0,"Lamar, Alabama, US",2042.738138,0.709220
669,1077.0,Lauderdale,Alabama,US,2020-09-10 04:29:01,34.901719,-87.656247,1490,25,0,1465.0,"Lauderdale, Alabama, US",1606.832814,1.677852


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,6361265,190859,2387479
India,4465863,75062,3471783
Brazil,4197889,128539,3611632


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,6361265,190859,2387479,3782927,2020-09-10 04:29:01,37.923092,-91.345036
India,4465863,75062,3471783,919018,2020-09-10 04:29:01,23.088275,81.806127
Brazil,4197889,128539,3611632,457718,2020-09-10 04:29:01,-12.669522,-48.480493
Russia,1037526,18080,854069,165377,2020-09-10 04:29:01,54.546312,62.120860
Peru,696190,30123,536959,129108,2020-09-10 04:29:01,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,747290,13983,0
Texas,667500,13923,0
Florida,652259,12110,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,747290,13983,0,733307,2020-09-10 04:29:01,37.843962,-120.728594
Texas,667500,13923,0,653577,2020-09-10 04:29:01,31.660643,-98.653069
Florida,652259,12110,0,640149,2020-09-10 04:29:01,28.940755,-82.700744
New York,441154,33019,0,408135,2020-09-10 04:29:01,42.544151,-75.474183
Georgia,287300,6128,0,281172,2020-09-10 04:29:01,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,249859,6090,0
Florida,Miami-Dade,162433,2740,0
Arizona,Maricopa,136324,3116,0
Illinois,Cook,132401,5100,0
Texas,Harris,112762,2363,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,249859,6090,0,243769,2020-09-10 04:29:01,34.308284,-118.228241,6037.0
Florida,Miami-Dade,162433,2740,0,159693,2020-09-10 04:29:01,25.611236,-80.551706,12086.0
Arizona,Maricopa,136324,3116,0,133208,2020-09-10 04:29:01,33.348359,-112.491815,4013.0
Illinois,Cook,132401,5100,0,127301,2020-09-10 04:29:01,41.841448,-87.816588,17031.0
Texas,Harris,112762,2363,0,110399,2020-09-10 04:29:01,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,38494,10406,46653,1261,2981,95,488007,44845,26373,29561,...,352451,1693,43893,54350,1049,26779,10,1989,12836,7298
2020-09-08,38520,10553,46938,1261,3033,95,500034,44953,26465,30081,...,354932,1712,44281,55563,1054,27363,10,1994,12952,7388
2020-09-09,38544,10704,47216,1301,3092,95,512293,45152,26524,30583,...,357613,1741,44930,56751,1059,27919,10,1999,13112,7429


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,1415,319,1562,53,120,3,10129,900,770,746,...,41643,45,352,436,35,184,1,573,295,210
2020-09-08,1418,321,1571,53,124,3,10405,903,781,747,...,41675,45,358,444,35,190,1,576,297,218
2020-09-09,1420,322,1581,53,126,3,10658,905,788,747,...,41683,45,366,452,35,192,1,576,300,222


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,30557,6186,32985,934,1215,91,357388,40121,22602,25300,...,1824,1466,41531,43753,853,17270,8,1201,11748,5455
2020-09-08,30715,6239,33183,934,1215,91,366590,40592,22723,25629,...,1827,1476,41594,44435,868,17779,8,1203,11787,5477
2020-09-09,31048,6284,33379,938,1245,91,382490,41023,22861,25764,...,1831,1478,42212,45318,890,18466,8,1209,11839,5542


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1348,1354,1345,1349,1355,1371,1377,1383,1385,1398
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,273,277,277,276,278,280,282,282,282,282
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1411,1424,1433,1443,1454,1467,1470,1479,1485,1490
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,463,466,469,486,488,501,502,505,506,510
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,3348,3395,3371,3384,3417,3466,3488,3511,3529,3553


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-07,132973,5793,0,205964,65727,740965,59464,53365,18249,49,...,165109,659041,55033,1651,1190,127516,77545,11575,81760,4032
2020-09-08,133606,5828,0,206045,66021,744344,59651,53782,18308,49,...,165754,662427,55359,1654,1191,128350,77545,11662,82477,4105
2020-09-09,134417,5891,0,206541,66406,747290,59891,53871,18308,49,...,166606,667500,55673,1656,1197,129229,78009,11814,83334,4152


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-07        1383    4586     617  557   1070     545    859    2593   
2020-09-08        1385    4609     617  562   1071     545    859    2602   
2020-09-09        1398    4639     618  564   1082     548    862    2615   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-07          857      367  ...     172      7      175       49   
2020-09-08          857      367  ...     173      7      180       50   
2020-09-09          858      371  ...     179      7      182       51   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-07            309   447   308          0      111     20  
2020-09-08            314   450   309          0      111     20  
2020-09-09            316   451   308          0      111     21  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-07,2276,42,0,5219,908,13758,1973,4468,609,0,...,1869,13697,423,58,17,2684,1953,248,1168,42
2020-09-08,2277,42,0,5221,917,13823,1973,4474,609,0,...,1896,13792,424,58,18,2686,1953,252,1168,42
2020-09-09,2285,42,0,5251,928,13983,1977,4474,609,0,...,1931,13923,427,58,18,2697,1978,255,1183,42


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-07          23      42       7    6     12      13     37      31   
2020-09-08          23      42       7    6     12      13     37      31   
2020-09-09          23      42       7    6     12      13     37      31   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-07           40       11  ...       1      1        1        1   
2020-09-08           40       11  ...       1      1        1        1   
2020-09-09           40       11  ...       1      1        1        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-07              2     1     2          0        6      0  
2020-09-08              2     1     2          0        6      0  
2020-09-09              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,0.002500,0.014725,0.006233,0.037860,0.005396,0.0,0.019246,0.001384,0.001976,0.009907,...,0.008443,0.008338,0.007020,0.019910,0.000000,0.024955,0.0,0.001007,0.004696,0.067427
2020-09-08,0.000675,0.014126,0.006109,0.000000,0.017444,0.0,0.024645,0.002408,0.003488,0.017591,...,0.007039,0.011223,0.008840,0.022318,0.004766,0.021808,0.0,0.002514,0.009037,0.012332
2020-09-09,0.000623,0.014309,0.005923,0.031721,0.019453,0.0,0.024516,0.004427,0.002229,0.016688,...,0.007554,0.016939,0.014656,0.021381,0.004744,0.020319,0.0,0.002508,0.012353,0.005550


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,0.002125,0.009494,0.003856,0.0,0.025641,0.0,0.027386,0.003344,0.010499,0.013587,...,0.000072,0.0,0.014409,0.018692,0.0,0.016575,0.0,0.001748,0.000000,0.019417
2020-09-08,0.002120,0.006270,0.005762,0.0,0.033333,0.0,0.027248,0.003333,0.014286,0.001340,...,0.000768,0.0,0.017045,0.018349,0.0,0.032609,0.0,0.005236,0.006780,0.038095
2020-09-09,0.001410,0.003115,0.006365,0.0,0.016129,0.0,0.024315,0.002215,0.008963,0.000000,...,0.000192,0.0,0.022346,0.018018,0.0,0.010526,0.0,0.000000,0.010101,0.018349


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,0.000655,0.013102,0.007329,0.006466,0.014190,0.0,0.023647,0.000798,0.006098,0.010262,...,0.000000,0.004798,0.007423,0.041589,0.046626,0.025352,0.0,0.000833,0.006339,0.020580
2020-09-08,0.005171,0.008568,0.006003,0.000000,0.000000,0.0,0.025748,0.011739,0.005354,0.013004,...,0.001645,0.006821,0.001517,0.015588,0.017585,0.029473,0.0,0.001665,0.003320,0.004033
2020-09-09,0.010842,0.007213,0.005907,0.004283,0.024691,0.0,0.043373,0.010618,0.006073,0.005267,...,0.002189,0.001355,0.014858,0.019872,0.025346,0.038641,0.0,0.004988,0.004412,0.011868


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-07,0.004981,0.005729,NaN,0.000957,0.005354,0.002854,0.003595,0.000000,0.011417,0.0,...,0.005989,0.001275,0.006824,0.001820,0.007621,0.005036,0.004014,0.013750,0.006983,0.000000
2020-09-08,0.004760,0.006042,NaN,0.000393,0.004473,0.004560,0.003145,0.007814,0.003233,0.0,...,0.003907,0.005138,0.005924,0.001817,0.000840,0.006540,0.000000,0.007516,0.008770,0.018105
2020-09-09,0.006070,0.010810,NaN,0.002407,0.005831,0.003958,0.004023,0.001655,0.000000,0.0,...,0.005140,0.007658,0.005672,0.001209,0.005038,0.006848,0.005984,0.013034,0.010391,0.011449


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-07      0.004357  0.003721  0.001623  0.003604  0.004695  0.000000   
2020-09-08      0.001446  0.005015  0.000000  0.008977  0.000935  0.000000   
2020-09-09      0.009386  0.006509  0.001621  0.003559  0.010271  0.005505   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2020-09-07      0.002334  0.013683  0.002339  0.005479  ...  0.000000    0.0   
2020-09-08      0.000000  0.003471  0.000000  0.000000  ...  0.005814    0.0   
2020-09-09      0.003492  0.004996  0.001167  0.010899  ...  0.034682    0.0   

Province_State                                                                \
Admin2          Sheridan  Sublette Sweetwater     Teton     Uinta Unassigned   
2020-09-07      0.000000  0.000000   0.000000  0.000000  0.000000        NaN   
2020-09-08      0.028571  0.020408   0.016181  0.006711  0.003247        NaN   
2020-09-09      0.011111  0.020000   0.006369  0.002222 -0.003236        NaN   

Province_State                  
Admin2         Washakie Weston  
2020-09-07          0.0   0.00  
2020-09-08          0.0   0.00  
2020-09-09          0.0   0.05  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-07,0.000000,0.0,NaN,-0.000383,0.015660,0.001966,0.000507,0.000000,0.0,NaN,...,0.002145,0.003149,0.002370,0.0,0.000000,0.002240,0.000000,-0.004016,0.000000,0.0
2020-09-08,0.000439,0.0,NaN,0.000383,0.009912,0.004725,0.000000,0.001343,0.0,NaN,...,0.014446,0.006936,0.002364,0.0,0.058824,0.000745,0.000000,0.016129,0.000000,0.0
2020-09-09,0.003513,0.0,NaN,0.005746,0.011996,0.011575,0.002027,0.000000,0.0,NaN,...,0.018460,0.009498,0.007075,0.0,0.000000,0.004095,0.012801,0.011905,0.012842,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-07         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-09-08         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2020-09-09         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-07          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-09-08          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-09-09          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-07            0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-08            0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-09            0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,0.001888,0.014371,0.006391,0.020212,0.010368,0.000332,0.019117,0.002482,0.002143,0.009649,...,0.007697,0.007998,0.006941,0.020973,0.000229,0.023294,3.676358e-25,0.001331,0.005504,0.035722
2020-09-08,0.001282,0.014249,0.006250,0.010106,0.013906,0.000166,0.021881,0.002445,0.002816,0.013620,...,0.007368,0.009611,0.007890,0.021646,0.002498,0.022551,1.838179e-25,0.001922,0.007271,0.024027
2020-09-09,0.000952,0.014279,0.006086,0.020914,0.016679,0.000083,0.023199,0.003436,0.002523,0.015154,...,0.007461,0.013275,0.011273,0.021513,0.003621,0.021435,9.190896e-26,0.002215,0.009812,0.014788


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,0.001651,0.012579,0.004542,1.432804e-10,0.016941,1.129797e-44,0.021601,0.003279,0.013357,0.007187,...,0.000115,0.001445,0.014099,0.018912,0.001243,0.021932,0.0,0.001087,0.002771,0.010146
2020-09-08,0.001886,0.009424,0.005152,7.164020e-11,0.025137,5.648984e-45,0.024425,0.003306,0.013821,0.004264,...,0.000442,0.000722,0.015572,0.018630,0.000621,0.027271,0.0,0.003161,0.004775,0.024121
2020-09-09,0.001648,0.006270,0.005759,3.582010e-11,0.020633,2.824492e-45,0.024370,0.002761,0.011392,0.002132,...,0.000317,0.000361,0.018959,0.018324,0.000311,0.018898,0.0,0.001581,0.007438,0.021235


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-07,0.006516,0.017357,0.007563,0.004590,0.013712,0.000044,0.024751,0.004969,0.006747,0.010596,...,0.007000,0.004339,0.008685,0.025408,0.034261,0.020106,2.154116e-27,0.002690,0.004697,0.011532
2020-09-08,0.005843,0.012963,0.006783,0.002295,0.006856,0.000022,0.025249,0.008354,0.006050,0.011800,...,0.004322,0.005580,0.005101,0.020498,0.025923,0.024789,1.077058e-27,0.002178,0.004009,0.007783
2020-09-09,0.008343,0.010088,0.006345,0.003289,0.015774,0.000011,0.034311,0.009486,0.006062,0.008534,...,0.003256,0.003468,0.009979,0.020185,0.025634,0.031715,5.385290e-28,0.003583,0.004210,0.009825


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-07,0.005846,0.010890,NaN,0.001757,0.008162,0.004252,0.004795,0.000323,0.009437,7.108417e-54,...,0.008134,0.003363,0.007326,0.001751,0.008994,0.006931,0.004885,0.014665,0.009521,0.003898
2020-09-08,0.005303,0.008466,NaN,0.001075,0.006317,0.004406,0.003970,0.004068,0.006335,3.554208e-54,...,0.006020,0.004250,0.006625,0.001784,0.004917,0.006735,0.002443,0.011091,0.009145,0.011002
2020-09-09,0.005687,0.009638,NaN,0.001741,0.006074,0.004182,0.003997,0.002862,0.003168,1.777104e-54,...,0.005580,0.005954,0.006148,0.001497,0.004978,0.006792,0.004213,0.012062,0.009768,0.011226


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-07      0.005128  0.004832  0.000598  0.005823  0.005408  0.001263   
2020-09-08      0.003287  0.004924  0.000299  0.007400  0.003171  0.000632   
2020-09-09      0.006337  0.005716  0.000960  0.005479  0.006721  0.003068   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-07      0.003727  0.012228  0.001604  0.005784  ...  0.001079   
2020-09-08      0.001863  0.007849  0.000802  0.002892  ...  0.003446   
2020-09-09      0.002678  0.006423  0.000984  0.006896  ...  0.019064   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2020-09-07      8.245309e-08  0.013748  0.004585   0.002398  0.004989   
2020-09-08      4.122655e-08  0.021159  0.012497   0.009289  0.005850   
2020-09-09      2.061327e-08  0.016135  0.016248   0.007829  0.004036   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2020-09-07      0.001071  -0.495957  0.002991  0.013007  
2020-09-08      0.002159  -0.495957  0.001496  0.006504  
2020-09-09     -0.000539  -0.495957  0.000748  0.028252  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-07,0.002019,0.007485,NaN,0.002363,0.014954,0.003444,0.001515,0.000012,0.000856,NaN,...,0.004578,0.005848,0.003819,2.058615e-12,0.018294,0.002586,0.000543,0.004847,0.002354,0.003214
2020-09-08,0.001229,0.003743,NaN,0.001373,0.012433,0.004084,0.000757,0.000678,0.000428,NaN,...,0.009512,0.006392,0.003091,1.029308e-12,0.038559,0.001666,0.000271,0.010488,0.001177,0.001607
2020-09-09,0.002371,0.001871,NaN,0.003560,0.012214,0.007830,0.001392,0.000339,0.000214,NaN,...,0.013986,0.007945,0.005083,5.146538e-13,0.019279,0.002880,0.006536,0.011196,0.007010,0.000804


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                             \
Admin2           Autauga  Baldwin       Barbour      Bibb    Blount   
2020-09-07      0.000355  0.00504  1.596910e-07 -0.003638  0.006450   
2020-09-08      0.000178  0.00252  7.984550e-08 -0.001819  0.003225   
2020-09-09      0.000089  0.00126  3.992275e-08 -0.000909  0.001613   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-09-07      4.334883e-08  0.001736 -0.000212  0.001615  0.000054  ...   
2020-09-08      2.167442e-08  0.000868 -0.000106  0.000808  0.000027  ...   
2020-09-09      1.083721e-08  0.000434 -0.000053  0.000404  0.000014  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-09-07         0.0    0.0      0.0      0.0        0.0   0.0   0.0   
2020-09-08         0.0    0.0      0.0      0.0        0.0   0.0   0.0   
2020-09-09         0.0    0.0      0.0      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-07      -0.499985      0.0    NaN  
2020-09-08      -0.499985      0.0    NaN  
2020-09-09      -0.499985      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200909,AK,6790,385147,NaN,40.0,NaN,NaN,NaN,10.0,...,391937,0,0,992ac2cfe3dbd2bb5a7e3ae2aeca31f35babde50,0,0,0,0,0,NaN
1,20200909,AL,134417,863982,NaN,857.0,15339.0,NaN,1551.0,NaN,...,998399,8,202,6ebb05ac2b35296345d3b0f9b7e9b2f0989c3e23,0,0,0,0,0,NaN
2,20200909,AR,67255,716973,NaN,411.0,4574.0,NaN,NaN,82.0,...,784228,11,50,b5a4a23cbe9ad0a5bde893beec9abe0e49aafba0,0,0,0,0,0,NaN
3,20200909,AS,0,1571,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,b568d4827408ce320e19f678e9d3d4da52b89bf1,0,0,0,0,0,NaN
4,20200909,AZ,206541,1062656,NaN,658.0,21593.0,203.0,NaN,111.0,...,1269197,30,71,4786f4abd095b7eab136e6374af9f0299086bd3d,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200909,AK,6790.0,385147.0,NaN,40.0,NaN,NaN,NaN,10.0,...,391937,0,0,992ac2cfe3dbd2bb5a7e3ae2aeca31f35babde50,0,0,0,0,0,NaN
1,20200909,AL,134417.0,863982.0,NaN,857.0,15339.0,NaN,1551.0,NaN,...,998399,8,202,6ebb05ac2b35296345d3b0f9b7e9b2f0989c3e23,0,0,0,0,0,NaN
2,20200909,AR,67255.0,716973.0,NaN,411.0,4574.0,NaN,NaN,82.0,...,784228,11,50,b5a4a23cbe9ad0a5bde893beec9abe0e49aafba0,0,0,0,0,0,NaN
3,20200909,AS,0.0,1571.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,b568d4827408ce320e19f678e9d3d4da52b89bf1,0,0,0,0,0,NaN
4,20200909,AZ,206541.0,1062656.0,NaN,658.0,21593.0,203.0,NaN,111.0,...,1269197,30,71,4786f4abd095b7eab136e6374af9f0299086bd3d,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-09,AK,6790,385147,NaN,40.0,NaN,NaN,NaN,10.0,NaN,...,391937,0,0,992ac2cfe3dbd2bb5a7e3ae2aeca31f35babde50,0,0,0,0,0,NaN
2020-09-09,AL,134417,863982,NaN,857.0,15339.0,NaN,1551.0,NaN,854.0,...,998399,8,202,6ebb05ac2b35296345d3b0f9b7e9b2f0989c3e23,0,0,0,0,0,NaN
2020-09-09,AR,67255,716973,NaN,411.0,4574.0,NaN,NaN,82.0,586.0,...,784228,11,50,b5a4a23cbe9ad0a5bde893beec9abe0e49aafba0,0,0,0,0,0,NaN
2020-09-09,AS,0,1571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,b568d4827408ce320e19f678e9d3d4da52b89bf1,0,0,0,0,0,NaN
2020-09-09,AZ,206541,1062656,NaN,658.0,21593.0,203.0,NaN,111.0,NaN,...,1269197,30,71,4786f4abd095b7eab136e6374af9f0299086bd3d,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-09,AK,6790.0,385147.0,NaN,40.0,NaN,NaN,NaN,10.0,NaN,...,391937,0,0,992ac2cfe3dbd2bb5a7e3ae2aeca31f35babde50,0,0,0,0,0,NaN
2020-09-09,AL,134417.0,863982.0,NaN,857.0,15339.0,NaN,1551.0,NaN,854.0,...,998399,8,202,6ebb05ac2b35296345d3b0f9b7e9b2f0989c3e23,0,0,0,0,0,NaN
2020-09-09,AR,67255.0,716973.0,NaN,411.0,4574.0,NaN,NaN,82.0,586.0,...,784228,11,50,b5a4a23cbe9ad0a5bde893beec9abe0e49aafba0,0,0,0,0,0,NaN
2020-09-09,AS,0.0,1571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,b568d4827408ce320e19f678e9d3d4da52b89bf1,0,0,0,0,0,NaN
2020-09-09,AZ,206541.0,1062656.0,NaN,658.0,21593.0,203.0,NaN,111.0,NaN,...,1269197,30,71,4786f4abd095b7eab136e6374af9f0299086bd3d,0,0,0,0,0,NaN
